# Microblaze 서브시스템 프로그래밍


PYNQ의 Base I/O overlay에서는 다양한 커넥터에 부착된 주변 장치들을 제어하기 위해 Microblaze가 사용되며, 이는 외부에서 컴파일된 기존 프로그램이나 주피터 내에서 구현된다. 이 장에서는 Microblaze가 어떻게 Jupyter, Python과 통합되는지 설명한다.

Microblaze는 C언어로 구현되어 있다. PYNQ framwork는 jupyter에서 작성한 C코드를 컴파일하여 Microblaze에 로딩한 후 실행 및 상호작용하는 메커니즘을 제공한다.

첫 번째 단계는 overlay를 로드하는 것이다.

In [ ]:
from pynq.overlays.base import BaseOverlay

base = BaseOverlay('base.bit')

이제 C코드를 쓸 수 있다. `%%microblaze` magic은 코드를 쓸 수 있는 환경을 제공하는데, 이 코드를 목표로 삼고자 하는 Microblaze라는 하나의 인자만을 요한다. 첫 번째 예제는 덧셈 후 결과 값을 반환받는 것이다.

In [ ]:
%%microblaze base.ARDUINO

int add(int a, int b) {
    return a + b;
}

magic에서 정의한 이 함수는 이제 파이썬에서 다른 함수와 상호 작용할 수 있다.

In [ ]:
add(4,6)

## Data Motion

 Python binding의 주 목적은 host와 slave 프로세서 간 데이터를 전송하도록 하는 것이다. 단순한 경우에 primitive C type을 함수 매개 변수로 사용할 수 있고, 결과 값을 반환받을 수 있다. 또한, Python에서의 값은 필요한만큼 자동으로 변환된다.

In [ ]:
%%microblaze base.ARDUINO

float arg_passing(float a, char b, unsigned int c) {
    return a + b + c;
}

In [ ]:
arg_passing(1, 2, 3)

배열은 두 가지 방법으로 전달될 수 있다. `void`가 아닌 다른 유형이 제공되면 데이터는 Microblaze에 저장되고, non-`const`라면 이 데이터도 다시 복사된다. 반복적이고 수정 가능한 객체는 함수의 인자로 사용될 수 있다.

In [1]:
%%microblaze base.ARDUINO

int culm_sum(int* val, int len) {
    int sum = 0;
    for (int i = 0; i < len; ++i) {
        sum += val[i];
        val[i] = sum;
    }
    return sum;
}

UsageError: Cell magic `%%microblaze` not found.


In [ ]:
numbers = [i for i in range(10)]
culm_sum(numbers, len(numbers))
print(numbers)

Microblaze가 대량의 데이터를 전송하기 위한 호스트 프로세싱 시스템의 메모리에 직접적으로 접근할 수 있도록 하는 `void` pointer를 함수 인자에 준다. Python에서는 이러한 메모리 블록을 `Xlnk.cma_array`함수를 사용하여 할당해야하며, Python과 C 코드에서 사용된 type들이 허용되는지 확인해야 한다.

In [ ]:
%%microblaze base.ARDUINO

long long big_sum(void* data, int len) {
    int* int_data = (int*)data;
    long long sum = 0;
    for (int i = 0; i < len; ++i) {
        sum += int_data[i];
    }
    return sum;
}

In [ ]:
from pynq import Xlnk
allocator = Xlnk()

buffer = allocator.cma_array(shape=(1024 * 1024), dtype='i4')
buffer[:] = range(1024*1024)

big_sum(buffer, len(buffer))

## Debug 출력하기

PYNQ Microblaze 환경에서는 new `pyprintf` 함수를 통하여 Jupyter 또는 Python 콘솔에 직접 Debug 정보를 출력한다. 이 함수는 Python에서 `printf`와 `format` 같이 기능하며, 출력을 위해 format string과 변수들을 Python으로 다시 전달할 수 있게 한다. 여기서는 `%d` format specifier가 지원되나, 시간이 지남에 따라 이는 증가할 것이다.( In this release on the %d format specifier is supported but this will increase over time.(?))
먼저 `pyprintf`를 사용하려면 적절한 헤더를 포함해야 한다.

In [ ]:
%%microblaze base.ARDUINO
#include <pyprintf.h>

int debug_sum(int a, int b) {
    int sum = a + b;
    pyprintf("Adding %d and %d to get %d\n", a, b, sum);
    return sum;
}

In [ ]:
debug_sum(1,2)

## 장기 실행 프로세스

지금까지 제시된 모든 예시는 결과가 나올 때까지의 Python 코드 blocking과 함께 Python 코드와 동기화되어 있었다. 일부 어플리케이션은 다른 함수에 의해 주기적으로 퀴리되는 장기 실행 프로세스를 요구한다. 만약 C 함수가 `void`을 반환하면 Python 프로세스가 즉시 재개되어 함수를 스스로 실행한다.

장기 실행 프로세스가 활성화되어 있는 동안에는 다른 함수들을 실행할 수 있지만, pre-emptive multithreading이 없기 때문에 다른 대기중인 함수들(other queued functions to run)을 실행하도록 하기 위해 persistent 프로세스는 non-timing critical point에서 `yield`해야만 한다.

아래 예에서 간단한 counter 프로세스를 실행한 후 두 번째 함수를 통해 결과 값을 반환한다.

In [ ]:
%%microblaze base.ARDUINO
#include <yield.h>

static int counter = 0;

void start_counter() {
    while (1) {
        ++counter;
        yield();
    }
}

int counter_value() {
    return counter;
}

이제 counter를 가동할 수 있다.

In [ ]:
start_counter()

현재 값들을 확인한다.

In [ ]:
counter_value()

subsequent 함수가 호출될 때까지 output이 표시되지 않기 때문에 persistent 함수 내에서 `pyprintf`를 사용하는 것에는 약간의 제한이 있다. 그 사이에 buffer가 채워지면 프로세스가 deadlock 상태가 될 수 있다.
한 번에 하나의 persistent 프로세스만 호출할 수 있다(다른 프로세스가 시작되면 첫 번째 프로세스가 다시 시작될 때까지 차단됨). 이런식으로 너무 많은 프로세스가 쌓이면 stack overflow로 인해 정의되지 않은 결과가 발생할 수 있다.

## 클래스와 비슷한 객체 생성하기


C 코드에서는 `typedef`를 사용하여 Python에 pseudo 클래스를 만들 수 있다. `my_class`라는 `typedef`가 있다면 `my_class_`가 되는 모든 함수는 `my_class`와 연관되어 있는 것으로 간주한다. 그런 함수 중 하나가 `my_class`를 첫 번째 인자로 삼는다면 그것은 `self`와 동등해진다. `typedef`는 오직 primitive type만 지칭할 수 있다는 점에 유의하라. 아래 예제는 이를 %(나눗셈의 나머지)를 이용하여 보인다.    

In [ ]:
%%microblaze base.ARDUINO

typedef unsigned int mod_int;

mod_int mod_int_create(int val) { return val % 53; }
mod_int mod_int_add(mod_int lhs, int rhs) { return (lhs + rhs) % 53; }

이제 `create` 함수를 통하여 인스턴스를 만들고 반환된 객체에 대한 `add`를 호출할 수 있다. typedef 인스턴스의 기본값은 `.val` attribute에서 검색할 수 있다.

In [ ]:
a = mod_int_create(63)
b = a.add(4)
print(b)
print(b.val)

## MicroBlaze 인터페이스 코딩 가이드 라인


Microblaze subsystem과 Jupyter 통합의 경우 다음과 같은 것들이 지원되지 않으므로 몇 함수를 사용할 수 없다.

 * 여러 종류의 `struct` 또는 `union`
 * Pointers to pointers
 * returning pointers
 
모든 non `void*` 파라미터들은 스택에 전달되므로 이와 같은 방식으로 large array를 전달하면 stack overflow가 발생할 수 있음에 유의하라.